In [1]:
import vtk
import numpy as np
import pandas as pd
import time
import queue
import threading
import re
from PyQt5.QtWidgets import QApplication
from PyQt5.QtCore import QTimer
import sys

In [2]:
# Initialize vtk
tool_reader = vtk.vtkSTLReader() # read stl file
tool_reader.SetFileName("C:\\Users\\f007wsq\\Desktop\\tools_and_models\\tool_stl_models\\medtronic_chicken_foot.STL") # set the file name
tool_reader.Update() # update the reader

tool_mapper = vtk.vtkPolyDataMapper() # create a mapper, a mapper is used to map the data to the graphics primitives
tool_mapper.SetInputData(tool_reader.GetOutput()) # set the input to the reader

tool_actor = vtk.vtkActor() # create an actor, an actor is used to display the data
tool_actor.SetMapper(tool_mapper) # set the mapper to the actor

renderer = vtk.vtkRenderer() # create a renderer, a renderer is used to display the data
render_window = vtk.vtkRenderWindow() # create a render window, a render window is used to display the data
render_window.AddRenderer(renderer) # add the renderer to the render window
render_window.SetSize(2000, 1600) # set the size of the render window

render_window_interactor = vtk.vtkRenderWindowInteractor() # create a render window interactor, a render window interactor is used to interact with the data
render_window_interactor.SetRenderWindow(render_window) # set the render window to the render window interactor

renderer.AddActor(tool_actor) # add the actor to the renderer
renderer.SetBackground(1, 1, 1) # set the background of the renderer

In [3]:
# Creat Coordinate System
axes = vtk.vtkAxesActor()
axes.SetTotalLength(200, 200, 200)  # 设置坐标轴的长度
axes.SetAxisLabels(True)

# Setting Axis Labels
axes.GetXAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetYAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()
axes.GetZAxisCaptionActor2D().GetTextActor().GetPositionCoordinate().SetCoordinateSystemToNormalizedDisplay()

# Setting Axix Colors
axes.GetXAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(1, 0, 0)  # X轴为红色
axes.GetYAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 1, 0)  # Y轴为绿色
axes.GetZAxisCaptionActor2D().GetTextActor().GetProperty().SetColor(0, 0, 1)  # Z轴为蓝色

# Adding Coordinates to renderer
renderer.AddActor(axes)

# Setting Camera Position
renderer.GetActiveCamera().SetPosition(-1000, 1000, -5000)  # 摄像机位置
renderer.GetActiveCamera().SetFocalPoint(0, 0, 0)  # 摄像机焦点
renderer.GetActiveCamera().SetViewUp(-1, 0, 0)  # 设置“上”方向
renderer.GetActiveCamera().SetClippingRange(1, 3000)  # 设置裁剪范围

In [4]:
# Polaris Data Streaming Thread
polaris_matrix = np.eye(4) # Initialize polaris matrix
frame_idx = 0 # Initialize frame index
# Thread-safe queue
data_queue = queue.Queue() # Initialize data queue

def read_polaris_data(recording, frame_idx):
    row = recording.iloc[frame_idx]
    tracking = row[3]  # Extract tracking data from the recorded data
    # Extract numbers from the tracking data
    numbers = [float(x) if x != "nan" else np.nan for x in re.findall(r"nan|[-+]?\d*\.\d+e[+-]?\d+|[-+]?\d*\.\d+|\d+", tracking)]
    tracking_matrix = np.array(numbers).reshape(4, 4)

    return tracking_matrix

def polaris_thread():
    global polaris_matrix
    global frame_idx
    while True:
        # Read polaris data
        new_matrix = read_polaris_data(recording, frame_idx)
        # Put the data into the queue
        data_queue.put(new_matrix)
        # Increment the frame index
        frame_idx += 1
        # Sleep for 1/400 seconds
        time.sleep(1/400)


In [5]:
# VTK Update Thread
def update_pose():
    global polaris_matrix

    if not data_queue.empty():
        # Get the latest data from the queue
        polaris_matrix = data_queue.get() 
        # Create a transform
        transform = vtk.vtkTransform()
        # Set the matrix of the transform
        transform.SetMatrix(polaris_matrix.flatten()) 
        # Set the user transform of the actor
        tool_actor.SetUserTransform(transform)
        # Render the window
        render_window.Render()

In [ ]:
# Read the recording data
recording = pd.read_csv('recording.csv', header=None)
# Start the polaris thread
tracking_thread = threading.Thread(target=polaris_thread, daemon=True)
tracking_thread.start()

# Initialize the application
app = QApplication(sys.argv)
# Create a timer
timer = QTimer()
# Connect the timer to the update_pose function
timer.timeout.connect(update_pose)
# Start the timer
timer.start(1) 

# Render the window
render_window.Render()
render_window_interactor.Initialize()
render_window_interactor.Start()
